In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import cv2
import torch

import numpy as np
import matplotlib.pylab as plt
import torch.optim as optim
import torch.nn.functional as F

from models.VQVAE2 import VQVAE2
from models.VQVAE import VQVAE
from config import setSeed, getConfig
from customLoader import MinecraftData, LatentBlockDataset

from pprint import pprint
from os.path import join
from pathlib import Path

from torchvision.utils import make_grid
from torchvision.transforms import transforms

from torch.utils.data import DataLoader

In [5]:
conf = {
  "experiment": "test_2",
  "environment": "MineRLNavigate-v0",
  "batch_size": 256,
  "num_training_updates": 25000,
  "learning_rate": 0.001,
  "split": 0.9,
  "vqvae": {
    "num_hiddens": 128,
    "num_residual_hiddens": 32,
    "num_residual_layers": 2,
    "embedding_dim": 64,
    "num_embeddings": 512,
    "commitment_cost": 0.25,
    "decay": 0.99
  },
  "pixelcnn": {
    "epochs": 100,
    "save": "no",
    "log_interval": 100,
    "lr": 0.0003,
    "img_dim": 16,
    "batch_size": 32,
    "num_hiddens": 64,
    "num_layers": 15,
    "gen_samples": "no"
  }
}

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
vqvae = VQVAE(**conf['vqvae']).to(device)
vqvae2 = VQVAE2().to(device)

In [12]:
vqvae

VQVAE(
  (_encoder): Encoder(
    (_conv_1): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (_conv_2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (_conv_3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (_residual_stack): ResidualStack(
      (_layers): ModuleList(
        (0): Residual(
          (_block): Sequential(
            (0): ReLU(inplace=True)
            (1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): ReLU(inplace=True)
            (3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
        (1): Residual(
          (_block): Sequential(
            (0): ReLU(inplace=True)
            (1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (2): ReLU(inplace=True)
            (3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          )
        )
      )


In [13]:
vqvae2

VQVAE2(
  (enc_b): Encoder(
    (blocks): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ResBlock(
        (conv): Sequential(
          (0): ReLU(inplace=True)
          (1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (6): ResBlock(
        (conv): Sequential(
          (0): ReLU(inplace=True)
          (1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (2): ReLU(inplace=True)
          (3): Conv2d(32, 128, kernel_size=(1, 1), stride=(1, 1))
        )
      )
      (7): ReLU(inplace=True)
    )
  )
  (enc_t): Encoder(
    (blocks): Sequential(
 

In [27]:
lst = [1, 5, 7, 13, 22, 15, 26, 64, 34, 72, 52, 14]

In [28]:
for i, value in enumerate(lst):
    if value % 3 == 0:
        print(f"Index: {i}, value: {value}")

Index: 5, value: 15
Index: 9, value: 72
